In [1]:
import numpy as np
import pandas as pd
from scipy.spatial import distance
import time

### Helper Functions

#### RKF Functions

In [ ]:
def readMeasurementsRKF():

    with open('2PeopleCentroids.csv', newline='') as csvfile:
        data = list(csv.reader(csvfile))

    centroidFramesCartesianMeasurement = list()
    headerFound = False
    for rowIndex in range(0, len(data)):
        row = data[rowIndex]
        if row[0] == 'X' and row[1] == 'Y' and row[2] == 'CentroidNumber':
            if headerFound:
                centroidFramesCartesianMeasurement.append(frame)
                frame = np.array([])
            else:
                headerFound = True
                frame = np.array([])
        elif headerFound:
            if (not((row[0]) and (row[1]))):
                centroidFramesCartesianMeasurement.append(frame)
                headerFound = False
            else:
                X = float(row[0])
                Y = float(row[1])
                CentroidNumber = int(row[2])
                if(np.size(frame) ==0):
                    frame = np.array([[X],[Y]]) #transposed structure
                else:
                    frameTemp= np.array([[X],[Y]])
                    frame = np.hstack((frame,frameTemp))

    for i in range(0,len(centroidFramesCartesianMeasurement)):
        if(np.size(centroidFramesCartesianMeasurement[i]) == 0):
            centroidFramesCartesianMeasurement[i] = np.reshape(centroidFramesCartesianMeasurement[i],(0,0)) #Force dimensions for empty arrays

    return(centroidFramesCartesianMeasurement)

In [ ]:
def predictRKF(x, P, A, Q): #predict function
    xpred = np.matmul(A,x)
    Ppred = np.matmul(A,P)
    Ppred = np.matmul(Ppred,np.transpose(A)) + Q
    return(xpred, Ppred)

In [ ]:
def innovationRKF(xpred, Ppred, z, H, R): #innovation function
    nu = z - np.matmul(H,xpred)
    S = np.matmul(H,Ppred)
    S = R + np.matmul(S, np.transpose(H))
    return(nu, S)

In [ ]:
def innovateUpdateRKF(xpred, Ppred, nu, S, H):
    K = np.matmul(Ppred, np.transpose(H))
    K = np.matmul(K,np.linalg.inv(S)) #check inverse function
    xnew = xpred + np.matmul(K,nu)
    Pnew = np.matmul(K,S)
    Pnew = Ppred - np.matmul(Pnew,np.transpose(K)) 
    return(xnew, Pnew)

In [ ]:
def cartesianToPolarRKF(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)

In [2]:
def dataAssociationRKF(centroidPred, rthetacentroid):
    rthetacentroidCurrent = rthetacentroid
    centpredCol = np.size(centroidPred,1)
    rthetaCol = np.size(rthetacentroid,1)

    for i in list(range(0,centpredCol)):
        r1 = centroidPred[0][i]
        r2 = rthetacentroid[0]
        theta1 = centroidPred[2][i] #HARDCODE: theta is 2nd index
        theta2 = rthetacentroid[1]
        temp = np.sqrt(np.multiply(r1,r1) + np.multiply(r2,r2) - np.multiply(np.multiply(np.multiply(2,r1),r2),np.cos(theta2-theta1)))
        if(i==0):
            minDist = temp
        else:
            minDist = np.vstack((minDist,temp))

    currentFrame = np.empty((2,max(centpredCol,rthetaCol)))
    currentFrame[:] = np.nan

    minDist = np.reshape(minDist, (centpredCol,rthetaCol))
    minDistOrg = minDist

    for i in list(range(0,min(centpredCol,rthetaCol))):
        if((np.ndim(minDist)) == 1):
            minDist = np.reshape(minDist,(rthetaCol,1))
            minDistOrg = np.reshape(minDistOrg,(rthetaCol,1))
        val = np.min(minDist)
        resultOrg = np.argwhere(minDistOrg == val)
        result = np.argwhere(minDist == val)
        minRowOrg = resultOrg[0][0]
        minColOrg = resultOrg[0][1]
        minRow = result[0][0]
        minCol = result[0][1]
        currentFrame[:,minRowOrg] = rthetacentroid[:,minColOrg]
        minDist = np.delete(minDist,minRow,0)
        minDist = np.delete(minDist,minCol,1)
        rthetacentroidCurrent = np.delete(rthetacentroidCurrent,minCol,1)

    index = 0
    if (rthetacentroidCurrent.size != 0): #check indexing
        for i in list(range(centpredCol,rthetaCol)):
            currentFrame[:,i] = rthetacentroidCurrent[:,index]
            index += 1 

    return(currentFrame)

In [ ]:
def RKF(centroidX,centroidP,A,H,P,Q,R,):
    rawxycentroidData = readMeasurementsRKF()
    
    for currentrawxycentroidData in rawxycentroidData:

        xytransposecentroidData = currentrawxycentroidData
        rthetacentroidData=xytransposecentroidData
        if (xytransposecentroidData.size != 0): 
            [rthetacentroidData[0,:],rthetacentroidData[1,:]] = cart2pol(xytransposecentroidData[0,:],xytransposecentroidData[1,:])
        if((rthetacentroidData.size != 0)):
            currentFrame = dataAssociationRKF(centroidX, rthetacentroidData)
            addittionalCentroids = (np.size(rthetacentroidData,1)-np.size(centroidX,1))
            if(addittionalCentroids>0):
                centroidX = np.pad(centroidX, ((0,0),(0,addittionalCentroids)), 'constant') #initialises previous iteration to zer
                for newFrameIndex in list((range(0, addittionalCentroids))):
                    centroidP.extend([P])
            for currentFrameIndex in list((range(0,np.size(currentFrame,1)))):
                if(not(np.isnan(currentFrame[0,currentFrameIndex]))):
                    [xpred, Ppred] = predict(centroidX[:,currentFrameIndex], centroidP[currentFrameIndex], A, Q)
                    [nu, S] = innovation(xpred, Ppred, currentFrame[:, currentFrameIndex], H, R)
                    [centroidX[:,currentFrameIndex],  centroidP[currentFrameIndex]] = innovation_update(xpred, Ppred, nu, S, H)
                else:
                    [centroidX[:,currentFrameIndex], Ppred] = predict(centroidX[:,currentFrameIndex], P, A, Q)
        else:
            for noFrameIndex in list((range(0,np.size(centroidX,1)))):
                [centroidX[:,noFrameIndex], Ppred] = predict(centroidX[:,noFrameIndex], P, A, Q)
        
        #Centroid 
        filteredPolarDf = pd.DataFrame(np.transpose(centroidX))
        filteredPolarDf.columns = ['FilteredRange', 'FilteredDoppler', 'FilteredTheta', 'FilteredAngularVelocity']
        filteredPolarDf['CentroidNumber'] = filteredPolarDf.index
        filteredPolarDf['FrameNumber'] = np.repeat(frameNumber, repeats=filteredPolarDf.shape[0])
        filteredPolarDf.to_csv('TrackingData.csv', mode='a', index=False)
    

#### DKF Functions

In [ ]:
def readMeasurementsDKF():
    #read in measurements from csv
    rawCentroidData = pd.read_csv('2PeopleCentroids.csv', header=None)
    #find headers and frames within headers
    #each header has the structure X, Y, CentroidNumber
    #below each header is the frame data
    centroidFramesCartesianMeasurement = list()
    headerFound = False
    for rowIndex in range(0, rawCentroidData.shape[0]):
        row = rawCentroidData.loc[rowIndex]
        if row[0] == 'X' and row[1] == 'Y' and row[2] == 'CentroidNumber':
            if headerFound: 
                #actual data was found last frame and this frame actual data is found again
                #past frame ended so add to list
                centroidFramesCartesianMeasurement.append(frame)
                frame = pd.DataFrame([])
            else:
                #header found
                headerFound = True
                frame = pd.DataFrame([])
            #data should be following
        elif headerFound:
            if np.isnan(np.float(row[0])) and np.isnan(np.float(row[1])):
                #empty row 
                centroidFramesCartesianMeasurement.append(frame)
                headerFound = False
                #only time its going to be NaN if the frame is completely empty
            else:
                #actual data
                X = np.float(row[0])
                Y = np.float(row[1])
                CentroidNumber = int(row[2])
                if len(frame) == 0:
                    frame = pd.DataFrame({'X':X, 'Y':Y, 'CentroidNumber':CentroidNumber}, index=range(1))
                else:
                    data = pd.DataFrame({'X':X, 'Y':Y, 'CentroidNumber':CentroidNumber}, index=range(CentroidNumber,CentroidNumber+1))
                    frame = pd.concat([frame,data])
    
    return centroidFramesCartesianMeasurement

In [3]:
def predictStepDKF(x, A):
    xPred = np.matmul(A,x)
    return xPred

In [4]:
def innovationStepDKF(xPred,z, H):
    innovation = np.subtract(z, np.matmul(H, xPred)) #innovation
    return innovation

In [ ]:
def UpdateStepDKF(xPred, nu, K):
    xNew = np.add(xPred, np.matmul(K, innovation))
    return xNew

In [ ]:
#convert x,y to r,theta
def convertCartesianToPolarDKF(cartesianMeasurementCentroidDf):
    measurementR = np.sqrt(cartesianMeasurementCentroidDf['X']**2 + cartesianMeasurementCentroidDf['Y']**2)
    measurementTheta = np.arctan2(cartesianMeasurementCentroidDf['Y'],cartesianMeasurementCentroidDf['X'])
    polarCentroidDf = pd.DataFrame({'CentroidNumber':np.arange(0,max(cartesianMeasurementCentroidDf['CentroidNumber'])+1), 
                                   'MeasuredRange': measurementR,
                                   'MeasuredTheta': measurementTheta})
    polarCentroidDf.index = range(0, polarCentroidDf.shape[0])
    return polarCentroidDf

In [ ]:
def createDistanceMatrixDKF(rThetaMeasurement,previousFrame):
    #create distance matrix
    distanceMatrix = np.eye(previousFrame.shape[0], rThetaMeasurement.shape[0])
    for centroid in previousFrame['CentroidNumber']:
        prevCentroidInfo = previousFrame.loc[centroid]
        #find distances between a selected prev centroid and the new centroids
        distances = np.sqrt(np.repeat(np.square(prevCentroidInfo.FilteredRange), repeats=rThetaMeasurement.shape[0]) + \
        np.square(rThetaMeasurement['MeasuredRange']) - \
        2*np.repeat(prevCentroidInfo.FilteredRange, repeats=rThetaMeasurement.shape[0]) * \
        rThetaMeasurement['MeasuredRange'] * np.cos(rThetaMeasurement['MeasuredTheta']-prevCentroidInfo.FilteredTheta))
        #assign distances
        distanceMatrix[centroid,:] = distances
    return distanceMatrix

In [ ]:
def dataAssociationDKF(rThetaMeasurement, previousFrame):
    #create distance matrix
    distanceMatrix = createDistanceMatrixDKF(rThetaMeasurement,previousFrame)
    association = np.full((max(distanceMatrix.shape[0], distanceMatrix.shape[1]), 2), np.nan)

    #associate
    numberOfLoops = min(distanceMatrix.shape[0], distanceMatrix.shape[1])
    for loopIterator in range(0, numberOfLoops):
        previousCentroid, measuredCentroid = np.where(np.min(distanceMatrix) == distanceMatrix)
    #     rThetaMeasurement.at[measuredCentroid[0],'CentroidNumber'] = previousCentroid[0] #associate with old centroid 
        association[loopIterator,0]= previousCentroid[0] #fill association matrix
        association[loopIterator,1]= measuredCentroid[0] #fill association matrix
        distanceMatrix[previousCentroid[0], :] = np.Inf
        distanceMatrix[:,measuredCentroid[0]] = np.Inf

    if np.isnan(association).any(): #if any NaN's still in the association matrix - mismatch alert
        if distanceMatrix.shape[0] > distanceMatrix.shape[1]:
            #more predictions than observations
            unassociatedPredictions = [pred for pred in list(previousFrame['CentroidNumber']) if pred not in list(association[:,0])]
            association[np.isnan(association[:,0]),0] = unassociatedPredictions
        else:
            #more observations than predictions
            #find which observation has not been associated
            unassociatedObservations = [observation for observation in list(rThetaMeasurement['CentroidNumber']) if observation not in list(association[:,1])]
            association[np.isnan(association[:,1]),1] = unassociatedObservations
            
    associationDf = pd.DataFrame(association, columns=['Predicted', 'Measured'])
    associationDf.index = pd.Series(associationDf['Predicted'].values.astype(int))
    
    return associationDf

In [ ]:
def DKF(A, H, Q, R, Pc, Pd, S, K, Pupdate, frameNumber):
    #read in measurements
    centroidFramesCartesianMeasurement= readMeasurementsDKF()
    #filtered frames (polaar measurements)
    filteredFramesPolar = list()
    frameNumber = 0
    for centroidFrame in centroidFramesCartesianMeasurement:
        filteredFramePolar = np.array([])
        #first frame that is valid
        if len(filteredFramesPolar) == 0 and len(centroidFrame) > 0: #kalman filter initialization
            rThetaMeasurement = convertCartesianToPolarDKF(centroidFrame)
            for centroid in rThetaMeasurement['CentroidNumber']:
                x = np.expand_dims(np.array([0,0,0,0]), axis=1) #initialise 

                #predict
                xPred = predictStepDKF(x, A)
                #measurement
                centroidInformation = rThetaMeasurement.loc[centroid]
                measurement = np.expand_dims(np.array([centroidInformation.MeasuredRange,centroidInformation.MeasuredTheta]),
                                             axis=1)
                innovation = innovationStepDKF(xPred, measurement, H)
                #update
                xUpdate = UpdateStepDKF(xPred, innovation, K)
                xUpdate = np.vstack((xUpdate, centroid))
                #add value to array
                if len(filteredFramePolar) == 0:
                    filteredFramePolar = xUpdate
                else:
                    filteredFramePolar = np.vstack(filteredFramePolar,xUpdate)

        #valid frames
        elif len(centroidFrame) >= 0 and len(filteredFramesPolar) != 0:
            if len(centroidFrame) != 0:
                rThetaMeasurement = convertCartesianToPolarDKF(centroidFrame)
            else:
                rThetaMeasurement = pd.DataFrame([], columns=['CentroidNumber', 'MeasuredRange', 'MeasuredTheta'])
                #since measurement has nothing in it just predict and move on



            #perform data association
            associationDf = dataAssociationDKF(rThetaMeasurement, filteredFramesPolar[-1])
            previousFrame = filteredFramesPolar[-1]

            for centroidIndex in associationDf.index:

                #predict
                predictedCentroid = associationDf.loc[centroidIndex]['Predicted']
                if np.isnan(predictedCentroid):
                    x = np.expand_dims(np.array([0,0,0,0]), axis=1) #initialise 
                else:
                    xPred = previousFrame.loc[previousFrame['CentroidNumber'] == predictedCentroid]
                    x = np.expand_dims(xPred.values[0][:4], axis=1)

                xPred = predictStepDKF(x, A)
                #measurement
                associatedMeasuredCentroid = associationDf.loc[centroidIndex]['Measured']
                if not(np.isnan(associatedMeasuredCentroid)):
                    #if associated centroid exists
                    centroidInformation = rThetaMeasurement.loc[associatedMeasuredCentroid]
                    measurement = np.expand_dims(np.array([centroidInformation.MeasuredRange,centroidInformation.MeasuredTheta]),axis=1)
                    innovation = innovationStepDKF(xPred, measurement, H)
                    #update
                    xUpdate = UpdateStepDKF(xPred, innovation, K)
                else:
                    xUpdate = xPred
                #add in centroid number
                if np.isnan(predictedCentroid): #no predicted value new measurement
                    associatableCentroidNumber = int(np.nanmax(associationDf['Predicted']))
                    predictedCentroid = associatableCentroidNumber + 1
                xUpdate = np.vstack((xUpdate, predictedCentroid))
                #add value to array
                if len(filteredFramePolar) == 0:
                    filteredFramePolar = xUpdate
                else:
                    filteredFramePolar = np.hstack((filteredFramePolar,xUpdate))  

        else:
            frameNumber += 1
            continue
        frameNumber += 1

        filteredPolarDf = pd.DataFrame(np.transpose(filteredFramePolar))
        filteredPolarDf.columns = ['FilteredRange', 'FilteredDoppler', 'FilteredTheta', 'FilteredAngularVelocity', 'CentroidNumber']
        #convert centroid number to integer value this is to aid indexing
        filteredPolarDf['CentroidNumber'] = filteredPolarDf['CentroidNumber'].astype(int)
        filteredPolarDf['CentroidNumber'] = pd.Series(filteredPolarDf['CentroidNumber'].values)
        filteredPolarDf.index = filteredPolarDf['CentroidNumber'].values.astype(int)
        filteredPolarDf['FrameNumber'] = np.repeat(frameNumber, repeats=filteredPolarDf.shape[0])
        filteredFramesPolar.append(filteredPolarDf) 
        frameNumber += 1
        
        #write to csv 
        filteredPolarDf.to_csv('TrackingData.csv', mode='a', index=False)
    return filteredFramesPolar

In [ ]:
#initialize variables
deltaT = 50*10**-3 #50ms
frameNumber = 0
useDKF = False #if true using DKF if false then using RKF

if useDKF:
    #system matrix
    A = np.array([
        [1, deltaT, 0,0],
        [0,1,0,0],
        [0,0,1,deltaT],
        [0,0,0,1]
    ])

    #output matrix
    H = np.array([[1,0,0,0],
                  [0,0,1,0]])
    #covariance matrices
    Q = np.eye(4)
    R = np.ones(2).reshape(-1,1)
    Pc = np.eye(4)

    Pd = np.add(np.matmul(np.matmul(A,Pc), np.transpose(A)), Q) #prediction covariance
    S = np.add(R, np.matmul(np.matmul(H, Pd), np.transpose(H))) #innovation covariance
    K = np.matmul(Pd, np.matmul(np.transpose(H), np.linalg.inv(S))) #kalman gain
    Pupdate = np.subtract(Pd, np.matmul(np.matmul(K, S), np.transpose(K))) #update covariance

    filteredFramesPolar = list()
else:
    centroidX =np.zeros((4,1))
    centroidP = []
    delT = 0.0500
    A = np.array([[1,delT,0,0], [0,1,0,0], [0,0,1,delT], [0,0,0,1]])
    H = np.array([[1,0,0,0],[0,0,1,0]])
    P = np.identity(4);
    Q = np.multiply(0.9,np.identity(4))
    R = np.array([[1],[1]])
    centroidP.extend([P])
    

    